## 0. Notebook Parameters

---

### Notebook Settings

In [1]:
"""Google Colab settings"""
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

'Google Colab settings'

In [2]:
"""Jupyter settings"""
# Enable autoreload
%load_ext autoreload
%autoreload 2

# Pylint parameters
%config Completer.use_jedi = False

# Measure Runtime
# !pip install ipython-autotime
%load_ext autotime

time: 243 µs (started: 2021-03-02 06:14:18 +01:00)


### Imported Packages

#### Packages Usually Needed

In [3]:
"""Packages for manipulation of vectors, arrays, dataframes"""
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None) # Change display settings of pandas

"""Packages for cleaning dataset"""
import json
import string
import unicodedata

"""Packages for data visualization"""
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

time: 684 ms (started: 2021-03-02 06:14:18 +01:00)


#### Packages Specific to the Notebook

In [4]:
# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = ['covfefe', 'dont', 'e', 'g', 'kj', 'kcal',]

import matplotlib.pyplot as plt

time: 329 ms (started: 2021-03-02 06:14:19 +01:00)


## 1. Computational Extraction of N-grams

Method following the tutorial [From DataFrame to N-Grams](https://towardsdatascience.com/from-dataframe-to-n-grams-e34e29df3460)   
> A quick-start guide to creating and visualizing n-gram ranking using `nltk` for natural language processing.

### Import the Data

In [5]:
# Load the whole clean dataset
file ='../raw_data/ocr_labeled.csv'
off_df_base = pd.read_csv(file)

time: 3.29 s (started: 2021-03-02 06:14:19 +01:00)


In [6]:
# Deep copy of the dataframe to avoid to reload it
df = off_df_base.copy()  # Renew the DataFrame
#df = df[:10000].copy() # Sample of the dataset

time: 20.8 ms (started: 2021-03-02 06:14:23 +01:00)


In [7]:
# Brief look at the dataset
print(f"""Shape of the dataset: {df.shape}
""")
print(f"""Columns types of the dataset: 
{df.dtypes}
""")
print(f"""Head of the dataset:""")
display(df.head(2))

Shape of the dataset: (434896, 6)

Columns types of the dataset: 
barcode           int64
clean_text       object
fr_text          object
source           object
pnns_groups_1    object
pnns_groups_2    object
dtype: object

Head of the dataset:


,barcode,clean_text,fr_text,source,pnns_groups_1,pnns_groups_2
0,3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",/319/966/047/6748/1.json,fish meat eggs,meat
1,3199660219192,ker chant local decoupes de poulet conditionne par ldc bretagne lanfains rais classe a origine france prodtre et c offre speciale ne eleve prepare local dans notre region cuisdej plt sat kerchant fx origine france volaille francaise prixkg paids net prin apayer kg to a consommer jusqu au r conserver entre d c et c expedie le loc bretagne lanfains autrit pens,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTRIT\nPENS\n3 199660112 19192\n",/319/966/021/9192/1.json,fish meat eggs,meat


time: 10.3 ms (started: 2021-03-02 06:14:23 +01:00)


### Basic Cleaning

In [8]:
def basic_clean(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english'
                                           ) + nltk.corpus.stopwords.words(
                                            'french') + ADDITIONAL_STOPWORDS
    
    words = (unicodedata.normalize('NFKD', text)
            .encode('ascii', 'ignore')
            .decode('utf-8', 'ignore')
            .lower())
    # words = re.sub('\d+', '', words) # Remove numbers
    words = re.sub(r'[^\w\s]', '', words).split()
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

time: 824 µs (started: 2021-03-02 06:14:23 +01:00)


### Loop over the Function of Extraction

In [9]:
# Create a list of categories
categories_1 = list(df.pnns_groups_1.unique())

time: 20.1 ms (started: 2021-03-02 06:14:23 +01:00)


In [10]:
# Tokenize texts for the whole column 
words_global = basic_clean(''.join(str(df['clean_text'].tolist())))

# Create a dict of tokenized texts per categories
words_categories_1 = {k: basic_clean(''.join(
    str(df['clean_text'][df['pnns_groups_1'] == k].tolist()))
                                    ) for k in categories_1
                     }

time: 4min 50s (started: 2021-03-02 06:14:23 +01:00)


In [11]:
len(set(words_global))

826317

time: 806 ms (started: 2021-03-02 06:19:13 +01:00)


In [12]:
from functools import reduce

# Instanciate the df that will store the output of the whole loop
main_df_list = []

# Loop over the number of n-grams
for ng in range(6, 11):
    
    # Create an empty list to store all the dataframes before merging them
    df_list = []
    
    # Get the n-grams for all the categories
    n_grams_global = (pd.Series(nltk.ngrams(words_global, ng)).value_counts())
    df_list.append(
        pd.DataFrame({'pattern': n_grams_global.index, 
                      'global_occurences': n_grams_global.values,
                     })
    )
    
    # Get the n-grams for each category:
    for cat_k in words_categories_1.keys():
        n_grams_cat = (pd.Series(nltk.ngrams(words_categories_1[cat_k], ng)).value_counts())
        df_list.append(
            pd.DataFrame({'pattern': n_grams_cat.index,
                          cat_k: n_grams_cat.values,
                         })
        )
    
    # Concatenate the dataframes at the end of the iteration
    n_df = reduce(lambda left, right: pd.merge(left, right, on='pattern'), df_list)
    
    # Create a variable to specify the number of n-grams of the output 
    n_gram_size = str(ng) + '-grams'
    n_df.insert(0,'n_gram_size',n_gram_size)
    
    # Append the dataframe of this n-gram
    main_df_list.append(n_df)

# Concatenate the dataframes after the loop
ngrams_df = pd.concat(main_df_list, ignore_index= True)

time: 11min 27s (started: 2021-03-02 06:19:14 +01:00)


In [13]:
len(main_df_list[0])

2192

time: 2.35 ms (started: 2021-03-02 06:30:42 +01:00)


In [14]:
main_df_list[0].head()

,n_gram_size,pattern,global_occurences,fish meat eggs,sugary snacks,cereals and potatoes,milk and dairy products,fat and sauces,fruits and vegetables,salty snacks,beverages,composite foods
0,6-grams,"(matieres, grass, acides, gras, satures, glucides)",47446,6272,8497,4977,6522,2233,4269,3315,3166,8195
1,6-grams,"(grass, acides, gras, satures, glucides, sucre)",46059,5892,8253,4902,6334,2124,4125,3224,3146,8059
2,6-grams,"(energie, matieres, grass, acides, gras, satures)",38329,5106,6457,3747,5218,2219,3720,2564,2619,6679
3,6-grams,"(acides, gras, satures, glucides, sucre, fibre)",26184,1903,4255,4265,2454,1052,3210,1718,1956,5371
4,6-grams,"(glucides, sucre, fibre, alimentaires, proteines, sel)",25292,1807,4090,3870,2330,1073,3255,1760,1850,5257


time: 9.49 ms (started: 2021-03-02 06:30:42 +01:00)


In [15]:
main_df_list[-1].tail()

,n_gram_size,pattern,global_occurences,fish meat eggs,sugary snacks,cereals and potatoes,milk and dairy products,fat and sauces,fruits and vegetables,salty snacks,beverages,composite foods
759,10-grams,"(sature, glucides, sucre, fibre, alimentaires, proteines, sel, valeurs, nutritionnelles, moyennes)",15,2,3,1,2,1,1,3,2,2
760,10-grams,"(acides, gras, sature, glucides, sucre, fibre, alimentaires, proteines, sel, valeurs)",15,2,3,1,2,1,1,3,2,2
761,10-grams,"(acides, gras, satures, glucides, sucre, fibre, alimentaires, proteines, sel, con)",15,3,2,1,1,1,3,2,1,2
762,10-grams,"(acides, gras, satures, glucides, sucre, fibre, alimentaires, proteines, sel, er)",13,1,2,2,1,1,1,2,1,2
763,10-grams,"(acides, gras, saturesg, matieres, grass, glucides, sucre, fibre, alimentaires, proteines)",12,1,2,1,1,1,1,3,1,1


time: 18.9 ms (started: 2021-03-02 06:30:42 +01:00)


In [16]:
len(ngrams_df)

6921

time: 2.78 ms (started: 2021-03-02 06:30:42 +01:00)


In [17]:
# Save ngrams_df dataset to csv
ngrams_df.to_csv('../raw_data/ngrams_extracted_reduced_2.csv', index = False)

time: 46.2 ms (started: 2021-03-02 06:30:42 +01:00)


In [18]:
print("Hello")

Hello
time: 671 µs (started: 2021-03-02 06:30:42 +01:00)
